# Hold'em

Je commence toujours

## 1ere idée : Faire la même chose que la soeur

In [10]:
def jeuSoeur(jeu, listeCoups = ''):
    # Cas de base
    if len(jeu) <= 1:
        return listeCoups
    # Appel recursif
    else:
        # Je joue
        if jeu[0] >= jeu[-1]:
            jeu.pop(0)
            listeCoups = listeCoups + 'G'
        else:
            jeu.pop()
            listeCoups = listeCoups + 'D'
        # Ma soeur joue
        if jeu[0] >= jeu[-1]:
            jeu.pop(0)
        else:
            jeu.pop()
        return jeuSoeur(jeu, listeCoups)

In [13]:
import random
jeu = [random.randint(2,14) for i in range(8)]
print(jeu)
jeuSoeur(jeu)

[12, 7, 13, 12, 12, 3, 7, 4]


'GGGD'

L'algo n'est pas optimal.

**Exemple** : |6|14|4|3|

## 2eme idée : Minimiser le gain de l'adversaire

In [19]:
def jeuMiniSoeur(jeu, listeCoups = ''):
    # Cas de base
    if len(jeu) <= 1:
        return listeCoups
    # Appel recursif
    else:
        # Je joue
        if max(jeu[0], jeu[-2]) > max(jeu[1], jeu[-1]):
            jeu.pop(-1)
            listeCoups = listeCoups + 'D'
        else:
            jeu.pop()
            listeCoups = listeCoups + 'G'
        # Ma soeur joue
        if jeu[0] >= jeu[-1]:
            jeu.pop(0)
        else:
            jeu.pop()
        return jeuMiniSoeur(jeu, listeCoups)

In [20]:
import random
jeu = [random.randint(2,14) for i in range(8)]
print(jeu)
jeuMiniSoeur(jeu)

[2, 11, 7, 7, 14, 13, 5, 4]


'GDGG'

**Contre exemple** : |2|5|8|6|

## Exploration exhaustive

**On souhaite maximiser la métrique : (MonScore - SonScore)**

On commence par créer un programme qui liste toutes les combinaisons et les teste sur un jeu de cartes.
On retient celle donnant le score maximal.

In [51]:
def testTout(jeu):
    # liste les coups possibles
    nbCoups = len(jeu) // 2
    possibles = []
    for i in range(2 ** nbCoups):
        chemin = bin(i)[2:]
        while len(chemin) < nbCoups:
            chemin = '0' + chemin
        chemin = chemin.replace('0', 'G')
        chemin = chemin.replace('1', 'D')
        possibles.append(chemin)
    # test pour chaque possibilité les combinaisons sur le jeu et l'evalue
    # On essaie de récupérer le meilleur
    meilleur = (- sum(jeu), '')
    for possible in possibles:
        jeuTest = jeu.copy()
        poids = 0
        for i in range(nbCoups):
            # Mon coups
            if possible[i] == 'G':
                poids += jeuTest.pop(0)
            else:
                poids += jeuTest.pop()
            # Son coups
            if jeuTest[0] >= jeuTest[-1]: # si 1ere et derniere egale, elle prend gauche
                poids -= jeuTest.pop(0)
            else:
                poids -= jeuTest.pop()
        # si il reste une carte:
        if nbCoups * 2 != len(jeu):
            poids += jeuTest.pop(0)
        # On verifie le max
        if poids > meilleur[0]:
            meilleur = (poids, possible)
        # print(poids, possible)
    print('meilleure solution : ', meilleur)

In [52]:
import random
jeu = [random.randint(2,14) for i in range(4)]
print(jeu)
testTout(jeu)

[9, 6, 5, 11]
meilleure solution :  (3, 'DG')


Le programme est très couteux, on va donc tenter d'explorer l'arbre des possibilités.
On créé un algorithme de recherche en profondeur.

In [61]:
def jeuExplore(jeu, poids = 0, listeCoups = ''):
    # Cas de base
    if len(jeu) == 0: # si la liste est vide on arrête
        return (poids, listeCoups)
    elif len(jeu) == 1: # s'il reste une carte, elle est pour moi
        return (poids + jeu[0], listeCoups)
    # Appel recursif
    else:
        # A gauche
            # Mon coups
        jeu1, poids1, listeCoups1 = jeu.copy(), poids, listeCoups
        poids1 = poids1 + jeu1.pop(0)
        listeCoups1 = listeCoups1 + 'G'
            # Son coups : si 1ere et derniere egale, elle prend gauche
        if jeu1[0] >= jeu1[-1]:
            poids1 = poids1 - jeu1.pop(0)
        else:
            poids1 = poids1 - jeu1.pop()
            
        gauche = jeuExplore(jeu1, poids1, listeCoups1)
        
        # A droite
            # Mon coups
        jeu2, poids2, listeCoups2 = jeu.copy(), poids, listeCoups
        poids2 = poids2 + jeu2.pop()
        listeCoups2 = listeCoups2 + 'D'
            # Son coups : si 1ere et derniere egale, elle prend gauche
        if jeu2[0] >= jeu2[-1]:
            poids2 = poids2 - jeu2.pop(0)
        else:
            poids2 = poids2 - jeu2.pop()
            
        droite = jeuExplore(jeu2, poids2, listeCoups2)
        
        # On prend la plus grand poids
        if gauche[0] >= droite[0]:
            return gauche
        else:
            return droite

In [63]:
import random
jeu = [random.randint(2,14) for i in range(20)]
print(jeu)
testTout(jeu)
print(jeuExplore(jeu))

[9, 4, 7, 13, 13, 12, 12, 11, 12, 7, 8, 11, 11, 13, 5, 13, 9, 7, 10, 8]
meilleure solution :  (11, 'GDGDGGGGGG')
(11, 'GDGDGGGGGG')
